# Recap:
Electronic structure properties provide valuable insights into the characteristics of atomistic systems, such as molecules or materials. However, first-principle electronic structure calculations are computationally demanding, making them impractical for analyzing large structures or processes over extended time scales. Classical calculations on the other hand are fast but inaccurate and do not return meaningful results for most systems. Machine learning (ML) bridges this gap. Trained on ab-initio data, the machine learning model can be used to infer electronic structure properties. It can also be used for inverse design, circumventing tedeous high throughput screening.

<img src="figs/intro3.png"  />


In this tutorial, our focus lies on Machine Learning (ML) force fields, which find application in molecular dynamics (MD) simulations and structure relaxations. ML Force fields typically predict the energy of a system and calculate the atomic forces by differentiating the neural network output with respect to the atomic coordinates
$
\mathbf{\hat{F}} = - \nabla \hat{E}~.
$
The Loss function of the optimization task is then given by 
$$
\mathcal{L} = \rho \Vert E - \hat{E}\Vert^2 + (1-\rho) \left\Vert \mathbf{F} -  \mathbf{\hat{F}} \right\Vert~.
$$

In particular we are going to use the network architecture SchNet, which is based on the idea of continuous-filter convolutions. It can also be described in the framework of Message Passing or Graph Neural Networks.

<img src="figs/schnet_filters.png" />

The network architecture is depicted below

<img src="figs/schnet_blocks.png"/>

# Tutorial

To train SchNet we recommend using the software package SchNetPack:
https://github.com/atomistic-machine-learning/schnetpack

The readme file explains how to train a model using the CLI. Since we are using custom data, you must adapt the training configurations of SchNetPack. This can be achieved by creating your own experiment config file or by loading a config file provided in SchNetPack and override some configs using the CLI. 

Be aware that on our computation cluster it is strongly recommended to move the database file to the ```/tmp``` directory before training. Otherwise the training will be slowed down excessively. You can do so by specifying the ```data_workdir``` in the respective ```schnetpack.data.AtomsDataModule```. More information regarding the file system of the computation cluster can be found here: https://git.tu-berlin.de/ml-group/hydra/documentation/-/blob/main/README.md#tmp-and-temp

Once the training has finished you can create an ASE calculator based on the trained model. To this end you can use the ```spk.interfaces.SpkCalculator```. Atomic Simulation Environment (ASE) is an extensive tool to work with atomic systems: https://wiki.fysik.dtu.dk/ase/index.html

In [1]:
import torch
import schnetpack as spk
from ase.io import read
import numpy as np


model = torch.load("dummy_model")
cutoff_radius = model.representation.cutoff_fn.cutoff.item()

spk_calculator = spk.interfaces.SpkCalculator(
    model_file="dummy_model",
    neighbor_list=spk.transform.MatScipyNeighborList(cutoff=cutoff_radius),
    device="cuda",
)

at = read("./ethanol_data.xyz", index="0")
at.pbc = np.array([False, False, False])
at.calc = spk_calculator



INFO:schnetpack.interfaces.ase_interface:Loading model from dummy_model


Once you defined a calculator for your ```ase.Atoms``` object you can use it just like any other calculator provided by ASE. For example you can calculate the forces for the respective structure:

In [2]:
print("forces:\n", at.get_forces())

forces:
 [[ 3.25218122e-03  3.10400818e-02  4.70496531e-09]
 [ 1.31115625e-02 -1.37734916e-02 -1.93852645e-09]
 [ 7.93496426e-03  2.27089282e-02 -9.69263225e-10]
 [-4.61420370e-03 -2.06240360e-02  1.77281555e-02]
 [-4.61420370e-03 -2.06240378e-02 -1.77281592e-02]
 [-6.71029976e-03  1.19957030e-02  1.52832633e-02]
 [-6.71030255e-03  1.19957058e-02 -1.52832642e-02]
 [-1.69290788e-02 -1.04204854e-02  6.46175502e-10]
 [ 1.52793797e-02 -1.22983688e-02  8.07719377e-11]]


The ASE documentation will help you to run 
* MDs: https://wiki.fysik.dtu.dk/ase/ase/md.html and 
* Structure relaxations: https://wiki.fysik.dtu.dk/ase/ase/optimize.html

# Hints:

Most common reasons for exploding molecules or deformed structures:
* integration constant is too large
* your choice of units is inconsistent
* your temperature might be too large

Too low temperature on the other hand will slow down the exploration of the conformation space.

# Contact
* Jonas Lederer (MAR 4.059)
* jonas.lederer@tu-berlin.de